In [1]:
import pandas as pd

# Load the spreadsheet
file_path = 'Fedex working.xlsx'
xls = pd.ExcelFile(file_path)

# Load the specific sheet 'Sheet1' into a DataFrame
df_sheet1 = pd.read_excel(file_path, sheet_name='Sheet1')

In [2]:


# Extract relevant component columns for weight (W (KG)) and length (L (CM))
component_columns = df_sheet1.columns[df_sheet1.columns.str.contains('COMPONENT')]
weight_columns = df_sheet1.columns[df_sheet1.columns.str.contains('W \(KG\)')]
length_columns = df_sheet1.columns[df_sheet1.columns.str.contains('L \(CM\)')]

# Initialize columns for exceeding weight and length
df_sheet1['Exceed Weight Quantity'] = 0
df_sheet1['Exceed Length Quantity'] = 0

# Calculate exceed weight and length quantity
for index, row in df_sheet1.iterrows():
    exceed_weight_count = 0
    exceed_length_count = 0
    for w_col, l_col in zip(weight_columns, length_columns):
        if pd.notna(row[w_col]) and row[w_col] > 31:
            exceed_weight_count += 1
        if pd.notna(row[l_col]) and row[l_col] > 122:
            exceed_length_count += 1
    df_sheet1.at[index, 'Exceed Weight Quantity'] = exceed_weight_count
    df_sheet1.at[index, 'Exceed Length Quantity'] = exceed_length_count

# Determine the correct position for the new columns
rounded_vol_index = df_sheet1.columns.get_loc('Rounded Vol (KG)')
columns = list(df_sheet1.columns)

# Remove the 'Exceed Weight Quantity' and 'Exceed Length Quantity' columns from their current positions
columns.remove('Exceed Weight Quantity')
columns.remove('Exceed Length Quantity')

# Insert the new columns after 'Rounded Vol (KG)'
columns.insert(rounded_vol_index + 1, 'Exceed Weight Quantity')
columns.insert(rounded_vol_index + 2, 'Exceed Length Quantity')

# Reorder the DataFrame with the new column order
df_sheet1 = df_sheet1[columns]

# Display the first few rows to confirm the changes
df_sheet1.head()


,SKU,TOTAL WEIGHT,Vol (KG),Rounded Vol (KG),Exceed Weight Quantity,Exceed Length Quantity,US SHOPIFY,AU SHOPIFY,EU SHOPIFY,UK SHOPIFY,...,H (CM).18,Vol (KG).19,PRICE.18,COMPONENT 19,W (KG).18,L (CM).18,W (CM).19,H (CM).19,Vol (KG).20,PRICE.19
0,TR160S-4INV3-NZ,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,TR160S-4INV3-CA,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,TR160S-4INV3-UK,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,TR160S-4INV3-UK,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,TR160S-4INV3,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,TR160-4INV3-DB,87.65,25.55413,26.0,0,1,TR160-4INV3-DB,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


### New Conditions

In [4]:

# Extract relevant component columns for width (W (CM)), height (H (CM)), and calculate girth
width_columns = df_sheet1.columns[df_sheet1.columns.str.contains('W \(CM\)')]
height_columns = df_sheet1.columns[df_sheet1.columns.str.contains('H \(CM\)')]


# Initialize columns for exceeding width, height, and girth
df_sheet1['Exceed Width'] = 0
df_sheet1['Exceed Height'] = 0
df_sheet1['Exceed Girth'] = 0

# Calculate exceed width, height, and girth
for index, row in df_sheet1.iterrows():
    exceed_width = 0
    exceed_height = 0
    exceed_girth = 0
    for w_col, h_col, l_col in zip(width_columns, height_columns, length_columns):
        if pd.notna(row[w_col]) and row[w_col] > 76:
            exceed_width = 1
        if pd.notna(row[h_col]) and row[h_col] > 76:
            exceed_height = 1
        # Calculate girth: 2*(width + height) + length
        if pd.notna(row[w_col]) and pd.notna(row[h_col]) and pd.notna(row[l_col]):
            girth = 2 * (row[w_col] + row[h_col]) + row[l_col]
            if girth > 266:
                exceed_girth = 1
    df_sheet1.at[index, 'Exceed Width'] = exceed_width
    df_sheet1.at[index, 'Exceed Height'] = exceed_height
    df_sheet1.at[index, 'Exceed Girth'] = exceed_girth

# Determine the correct position for the new columns
rounded_vol_index = df_sheet1.columns.get_loc('Rounded Vol (KG)')
columns = list(df_sheet1.columns)

# Remove the new columns from their current positions if they exist
for new_col in ['Exceed Width', 'Exceed Height', 'Exceed Girth']:
    if new_col in columns:
        columns.remove(new_col)

# Insert the new columns after 'Rounded Vol (KG)'
columns.insert(rounded_vol_index + 1, 'Exceed Width')
columns.insert(rounded_vol_index + 2, 'Exceed Height')
columns.insert(rounded_vol_index + 3, 'Exceed Girth')

# Reorder the DataFrame with the new column order
df_sheet1 = df_sheet1[columns]

In [3]:
# Calculate the weight type based on the condition
df_sheet1['Weight Type'] = df_sheet1.apply(
    lambda row: 'Volumetric' if row['Vol (KG)'] > row['TOTAL WEIGHT'] else 'True Weight',
    axis=1
)

# Determine the correct position for the new 'Weight Type' column
exceed_length_index = df_sheet1.columns.get_loc('Exceed Length Quantity')
columns = list(df_sheet1.columns)

# Remove the 'Weight Type' column from its current position
columns.remove('Weight Type')

# Insert the 'Weight Type' column after 'Exceed Length Quantity'
columns.insert(exceed_length_index + 1, 'Weight Type')

# Reorder the DataFrame with the new column order
df_sheet1 = df_sheet1[columns]

# Display the first few rows to confirm the changes
df_sheet1.head()

,SKU,TOTAL WEIGHT,Vol (KG),Rounded Vol (KG),Exceed Weight Quantity,Exceed Length Quantity,Weight Type,US SHOPIFY,AU SHOPIFY,EU SHOPIFY,...,H (CM).18,Vol (KG).19,PRICE.18,COMPONENT 19,W (KG).18,L (CM).18,W (CM).19,H (CM).19,Vol (KG).20,PRICE.19
0,TR160S-4INV3-NZ,86.08,23.94445,24.0,0,0,True Weight,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,TR160S-4INV3-CA,86.08,23.94445,24.0,0,0,True Weight,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,TR160S-4INV3-UK,86.08,23.94445,24.0,0,0,True Weight,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,TR160S-4INV3,86.08,23.94445,24.0,0,0,True Weight,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,TR160-4INV3-DB,87.65,25.55413,26.0,0,1,True Weight,TR160-4INV3-DB,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


# something else 

In [17]:
# Load the new Excel file containing the SKU data
new_file_path = 'Export_2024-07-30_134629.xlsx'
new_df = pd.read_excel(new_file_path)

In [18]:
# Remove duplicates based on SKU from both datasets
df_sheet1 = df_sheet1.drop_duplicates(subset='SKU')
new_df_unique = new_df.drop_duplicates(subset='Variant SKU')

# Perform an inner join to include only the matching SKUs
merged_df_unique = pd.merge(new_df_unique, df_sheet1[['SKU', 'Exceed Weight Quantity', 'Exceed Length Quantity', 'Weight Type']],
                            left_on='Variant SKU', right_on='SKU', how='inner')

# Display the merged dataframe to confirm the addition of new columns
print(f"Total number of matching SKUs: {len(merged_df_unique)}")
print(merged_df_unique.head())

Total number of matching SKUs: 237
   Status      Variant ID         Variant SKU  Variant Weight  \
0  Active  42514735399106  TR8PRO-R2R-BUN2-EU           126.6   
1  Active  42514407063746  TR8PRO-R2R-BUN1-EU           126.6   
2  Active  47223492411737          TR80L-1-TK           110.0   
3  Active  47223428612441      TR8PRO-1-TK-EU             0.0   
4  Active  47582883807577        SA-09 BUNDLE             8.0   

  Variant Weight Unit  Variant Price  Variant Cost  \
0                  kg         4671.0           NaN   
1                  kg         3901.0           NaN   
2                  kg         8478.0           NaN   
3                  kg         6241.0           NaN   
4                  kg          338.0           NaN   

   Variant Metafield: custom.height [single_line_text_field]  \
0                                                0.0           
1                                                0.0           
2                                                0.0     

In [19]:
merged_df_unique

,Status,Variant ID,Variant SKU,Variant Weight,Variant Weight Unit,Variant Price,Variant Cost,Variant Metafield: custom.height [single_line_text_field],Variant Metafield: custom.width [single_line_text_field],Variant Metafield: custom.length [single_line_text_field],Variant Metafield: custom.weight [single_line_text_field],Variant Metafield: custom.volume [single_line_text_field],Variant Metafield: custom.volumetric_weight [single_line_text_field],round,Shipping price nl,SKU,Exceed Weight Quantity,Exceed Length Quantity,Weight Type
0,Active,42514735399106,TR8PRO-R2R-BUN2-EU,126.6,kg,4671.0,NaN,0.0,0.0,0.0,0.0,0.0,186.0,0.00064,141.314448,TR8PRO-R2R-BUN2-EU,0,0,Volumetric
1,Active,42514407063746,TR8PRO-R2R-BUN1-EU,126.6,kg,3901.0,NaN,0.0,0.0,0.0,0.0,0.0,144.0,0.48848,109.480800,TR8PRO-R2R-BUN1-EU,0,0,Volumetric
2,Active,47223492411737,TR80L-1-TK,110.0,kg,8478.0,NaN,0.0,0.0,0.0,0.0,0.0,117.0,0.00000,89.016312,TR80L-1-TK,0,1,Volumetric
3,Active,47223428612441,TR8PRO-1-TK-EU,0.0,kg,6241.0,NaN,0.0,0.0,0.0,0.0,0.0,111.0,0.00000,84.468648,TR8PRO-1-TK-EU,0,0,Volumetric
4,Active,47582883807577,SA-09 BUNDLE,8.0,kg,338.0,NaN,0.0,0.0,0.0,16.5,0.0,84.0,1.26990,64.004160,SA-09 BUNDLE,0,0,Volumetric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Active,46690063614297,FCOREBLU4,1.1,kg,769.0,484.58,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,NaN,FCOREBLU4,0,0,True Weight
233,Active,46690063647065,FCORERED4,1.1,kg,769.0,484.58,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,NaN,FCORERED4,0,0,True Weight
234,Draft,47807894061401,TR8-06-SEAT-G-EU,50.0,kg,999.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,NaN,TR8-06-SEAT-G-EU,0,0,Volumetric
235,Active,46749871997273,TR8PRO-EU,50.0,kg,659.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,NaN,TR8PRO-EU,0,0,True Weight


In [20]:
# Re-import new_df to ensure we're working with the original dataset state
new_df = pd.read_excel(new_file_path)

# Ensure that the SKUs are in the same format (as strings)
new_df['Variant SKU'] = new_df['Variant SKU'].astype(str)

# Merge to include all rows from new_df, adding columns from merged_df_unique where matches exist
updated_df = pd.merge(new_df, merged_df_unique[['Variant SKU', 'Exceed Weight Quantity', 'Exceed Length Quantity', 'Weight Type']],
                      on='Variant SKU', how='left')

# Display the resulting dataset
updated_df.head(), len(updated_df)

(   Status      Variant ID         Variant SKU  Variant Weight  \
 0  Active  42514735399106  TR8PRO-R2R-BUN2-EU           126.6   
 1  Active  42514407063746  TR8PRO-R2R-BUN1-EU           126.6   
 2  Active  47223492411737          TR80L-1-TK           110.0   
 3  Active  47223428612441      TR8PRO-1-TK-EU             0.0   
 4  Active  47582883807577        SA-09 BUNDLE             8.0   
 
   Variant Weight Unit  Variant Price  Variant Cost  \
 0                  kg         4671.0           NaN   
 1                  kg         3901.0           NaN   
 2                  kg         8478.0           NaN   
 3                  kg         6241.0           NaN   
 4                  kg          338.0           NaN   
 
    Variant Metafield: custom.height [single_line_text_field]  \
 0                                                0.0           
 1                                                0.0           
 2                                                0.0           
 3        

In [21]:
# Drop any existing 'TOTAL WEIGHT' in the updated_df to avoid confusion
if 'TOTAL WEIGHT' in updated_df.columns:
    updated_df = updated_df.drop(columns=['TOTAL WEIGHT'])

# Merging the TOTAL WEIGHT into the updated_df based on SKU
weights_update_df = pd.merge(new_df, df_sheet1[['SKU', 'TOTAL WEIGHT']], left_on='Variant SKU', right_on='SKU', how='left')
updated_df = pd.merge(updated_df, weights_update_df[['Variant SKU', 'TOTAL WEIGHT']], on='Variant SKU', how='left')

# Update Variant Weight where it is 0, using the TOTAL WEIGHT from weights_update_df
updated_df['Variant Weight'] = updated_df.apply(
    lambda row: row['TOTAL WEIGHT'] if row['Variant Weight'] == 0 and not pd.isna(row['TOTAL WEIGHT']) else row['Variant Weight'],
    axis=1
)

# Display the rows where Variant Weight was updated from 0
updated_rows = updated_df[(updated_df['Variant Weight'] == updated_df['TOTAL WEIGHT']) & (new_df['Variant Weight'] == 0)]
updated_rows, len(updated_rows)

(     Status      Variant ID         Variant SKU  Variant Weight  \
 3    Active  47223428612441      TR8PRO-1-TK-EU         40.3209   
 28    Draft  41410293924034           RS6-NS-EU          8.3000   
 29    Draft  41410293956802           RS6-S3-EU         20.8000   
 33    Draft  41410294087874           RS6-S4-EU         16.3000   
 37    Draft  41410294218946           RS6-S6-EU         26.9000   
 45   Active  47569447682393    TR160S-WMINV3-EU         78.5000   
 46   Active  47569447879001     TR160S-4INV3-EU         82.3000   
 54   Active  47569448075609    TR160S-DDINV3-EU         77.8000   
 57   Active  47569447616857    TR160S-WMPBNP-EU         64.5000   
 59   Active  47569448272217     TR160S-AINV3-EU         76.3000   
 67   Active  47569447649625    TR160S-WMPBAL-EU         60.5000   
 68   Active  47569447780697  TR160S-L-WMINV3-EU         68.7000   
 69   Active  47569447813465     TR160S-4PBNP-EU         68.3000   
 70   Active  47569447977305   TR160S-L-4INV3-EU

In [24]:
len(updated_df)

2990

### Just VALIDATION

In [5]:
import pandas as pd

# Load the spreadsheet
file_path = 'Fedex working.xlsx'
xls = pd.ExcelFile(file_path)

# Load the specific sheet 'Sheet1' into a DataFrame
df_sheet1 = pd.read_excel(file_path, sheet_name='Sheet1')

# Extract relevant component columns for weight (W (KG)) and length (L (CM))
component_columns = df_sheet1.columns[df_sheet1.columns.str.contains('COMPONENT')]
weight_columns = df_sheet1.columns[df_sheet1.columns.str.contains('W \(KG\)')]
length_columns = df_sheet1.columns[df_sheet1.columns.str.contains('L \(CM\)')]

# Initialize columns for exceeding weight and length
df_sheet1['Exceed Weight Quantity'] = 0
df_sheet1['Exceed Length Quantity'] = 0

# Calculate exceed weight and length quantity
for index, row in df_sheet1.iterrows():
    exceed_weight_count = 0
    exceed_length_count = 0
    for w_col, l_col in zip(weight_columns, length_columns):
        if pd.notna(row[w_col]) and row[w_col] > 31:
            exceed_weight_count += 1
        if pd.notna(row[l_col]) and row[l_col] > 122:
            exceed_length_count += 1
    df_sheet1.at[index, 'Exceed Weight Quantity'] = exceed_weight_count
    df_sheet1.at[index, 'Exceed Length Quantity'] = exceed_length_count

# Determine the correct position for the new columns
rounded_vol_index = df_sheet1.columns.get_loc('Rounded Vol (KG)')
columns = list(df_sheet1.columns)

# Remove the 'Exceed Weight Quantity' and 'Exceed Length Quantity' columns from their current positions
columns.remove('Exceed Weight Quantity')
columns.remove('Exceed Length Quantity')

# Insert the new columns after 'Rounded Vol (KG)'
columns.insert(rounded_vol_index + 1, 'Exceed Weight Quantity')
columns.insert(rounded_vol_index + 2, 'Exceed Length Quantity')

# Reorder the DataFrame with the new column order
df_sheet1 = df_sheet1[columns]

# Display the first few rows to confirm the changes
df_sheet1.head()

,SKU,TOTAL WEIGHT,Vol (KG),Rounded Vol (KG),Exceed Weight Quantity,Exceed Length Quantity,US SHOPIFY,AU SHOPIFY,EU SHOPIFY,UK SHOPIFY,...,H (CM).18,Vol (KG).19,PRICE.18,COMPONENT 19,W (KG).18,L (CM).18,W (CM).19,H (CM).19,Vol (KG).20,PRICE.19
0,TR160S-4INV3-NZ,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,TR160S-4INV3-CA,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,TR160S-4INV3-UK,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,TR160S-4INV3-UK,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,TR160S-4INV3,86.08,23.94445,24.0,0,0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,TR160-4INV3-DB,87.65,25.55413,26.0,0,1,TR160-4INV3-DB,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [6]:
# REMOVE UNNECESSARY COLUMNS

# Let's rmeove US SHOPIFY, AU SHOPIFY, EU SHOPIFY, and UK SHOPIFY columns
df_sheet1 = df_sheet1.drop(columns=['US SHOPIFY', 'AU SHOPIFY', 'EU SHOPIFY', 'UK SHOPIFY','NZ SHOPIFY'])

# Display the first few rows to confirm the changes

df_sheet1.head()

,SKU,TOTAL WEIGHT,Vol (KG),Rounded Vol (KG),Exceed Weight Quantity,Exceed Length Quantity,COMPONENT 1,W (KG),L (CM),W (CM),...,H (CM).18,Vol (KG).19,PRICE.18,COMPONENT 19,W (KG).18,L (CM).18,W (CM).19,H (CM).19,Vol (KG).20,PRICE.19
0,TR160S-4INV3-NZ,86.08,23.94445,24.0,0,0,TR160S-BLK-PART1,19.0,82.5,22.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,TR160S-4INV3-CA,86.08,23.94445,24.0,0,0,TR160S-BLK-PART1,19.0,82.5,22.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,TR160S-4INV3-UK,86.08,23.94445,24.0,0,0,TR160S-BLK-PART1,19.0,82.5,22.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,TR160S-4INV3,86.08,23.94445,24.0,0,0,TR160S-BLK-PART1,19.0,82.5,22.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,TR160-4INV3-DB,87.65,25.55413,26.0,0,1,TR1604-BLK-PART1,18.5,141.0,18.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
